In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

%matplotlib inline
import sklearn.datasets
from sklearn.datasets import load_iris
from sklearn import tree

df = pd.read_csv('../../datasets/Hitters.csv', index_col = False)

df['League']        = df.League.factorize()[0]
df['Division']      = df.Division.factorize()[0]
df['NewLeague']     = df.NewLeague.factorize()[0]

df.dropna(subset=['Salary'], inplace = True)
df.drop(['Player'], axis = 1, inplace = True)


X = df.drop(['Salary'], axis = 1)
y = df.Salary



# Preprocess

* Scale the salary and columns with StandardScaler

* Split test train
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=88)


# 1 Basic tree


In [19]:
# ----------------------------------------------------------------------------
#  Basic Decision tree - Baseline
# ----------------------------------------------------------------------------

np.random.seed(88)
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)

mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)


scores = cross_val_score(clf, X, y, cv=50, scoring='mean_squared_error')
print("Default Decision Tree scores: %0.2f (+/- %0.2f) "% (np.mean(scores),np.std(scores) ))


MSE: 108645.8246
Default Decision Tree scores: -150131.83 (+/- 159290.65) 


# Limit the tree

clf = DecisionTreeRegressor(max_depth = 3, min_samples_split = 10)

What MSE do you get?

# Bagging 

On default tree

from sklearn.ensemble import BaggingClassifier

        clf = DecisionTreeRegressor()
        bagging = BaggingRegressor(base_estimator = clf, n_estimators = 20, max_samples=0.5, bootstrap=True, oob_score = True)

MSE?


# Bagging on constrained tree

use the DecisionTreeRegressor(max_depth = 3, min_samples_split = 10) as the base regressor